In [92]:
!python -m pip install "pymongo[srv]"

from pymongo.mongo_client import MongoClient

client = MongoClient(port=27017)

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

/Users/tarun/Prog/assignment2-at2025-snow-flake-2512-main/.venv/bin/python: No module named pip
Pinged your deployment. You successfully connected to MongoDB!


Below, I used **referencing with ObjectIds** for products' relationships to suppliers and categories, as well as for employees' self-referencing manager relationship (ReportsTo). This approach was chosen because suppliers, categories, and employees are independent entities that exist separately and may be queried independently. For instance, a supplier might supply multiple products, and we may want to retrieve supplier information without always loading all their products. Using references avoids data duplication and maintains a single source of truth for each entity's information, similar to foreign keys in relational databases.

In contrast, I used **embedding** for the relationship between orders and order details. Each order document contains an embedded array of OrderDetails, where each detail includes the ProductID reference, quantity, unit price, and discount. This design choice reflects the fact that order details have no meaningful existence outside the context of their parent order - they are always created, read, and managed together with the order. Embedding improves query performance since a single database operation retrieves the complete order with all its line items, avoiding the need for joins or multiple queries.

The orders and products collections also reference customers and employees using ObjectIds rather than embedding complete customer or employee documents. This decision balances data consistency with performance: while orders are closely related to customers, a customer's information (such as address or contact details) may change over time, and maintaining these as references ensures any updates to customer data are immediately reflected across all their orders without requiring updates to multiple embedded copies.

In [93]:
from dataclasses import dataclass, fields, asdict
from typing import get_type_hints

@dataclass
class Categories:
    CategoryID: int
    CategoryName: str
    Description: str
    Picture: str

@dataclass
class Customers:
    CustomerID: str
    CompanyName: str
    ContactName: str
    ContactTitle: str
    Address: str
    City: str
    Region: str
    PostalCode: str
    Country: str
    Phone: str
    Fax: str

@dataclass
class Employees:
    EmployeeID: int
    LastName: str
    FirstName: str
    Title: str
    TitleOfCourtesy: str
    BirthDate: str
    HireDate: str
    Address: str
    City: str
    Region: str
    PostalCode: str
    Country: str
    HomePhone: str
    Extension: str
    Photo: str
    Notes: str
    ReportsTo: str
    PhotoPath: str

@dataclass
class OrderDetails:
    ProductID: int
    UnitPrice: float
    Quantity: float
    Discount: float

@dataclass
class Orders:
    OrderID: int
    CustomerID: str
    EmployeeID: int
    OrderDate: str
    RequiredDate: str
    ShippedDate: str
    ShipVia: int
    Freight: float
    ShipName: str
    ShipAddress: str
    ShipCity: str
    ShipRegion: str
    ShipPostalCode: str
    ShipCountry: str

@dataclass
class Products:
    ProductID: int
    ProductName: str
    SupplierID: int
    CategoryID: int
    QuantityPerUnit: str
    UnitPrice: float
    UnitsInStock: int
    UnitsOnOrder: int
    ReorderLevel: int
    Discontinued: int

@dataclass
class Suppliers:
    SupplierID: int
    CompanyName: str
    ContactName: str
    ContactTitle: str
    Address: str
    City: str
    Region: str
    PostalCode: str
    Country: str
    Phone: str
    Fax: str
    HomePage: str


def convert(value, target_type):
    if isinstance(value, target_type):
        return value
    if target_type is int:
        return int(value)
    if target_type is float:
        return float(value)
    if target_type is str:
        return str(value)
    return value

class RowOperations:
    def __init__(self, collection_name):
        self.collection = collection_name

    def map_name_to_dc(self):
        name_to_dc = {
            "categories" : Categories,
            "customers": Customers,
            "employees": Employees,
            "orders": Orders,
            "products": Products,
            "suppliers": Suppliers,
            "order_details": OrderDetails
        }
        return name_to_dc[self.collection]


    def dataclass_from_sequence(self, cls, seq):
        cls_fields = fields(cls)
        hints = get_type_hints(cls)

        converted = {}

        for field, value in zip(cls_fields, seq):
            target_type = hints[field.name]
            converted[field.name] = convert(value, target_type)

        return asdict(cls(**converted))

    def converted(self, row):
        cls = self.map_name_to_dc()
        return self.dataclass_from_sequence(cls, row)



In [94]:
db = client['Northwind']

categories = db['categories']
suppliers = db['suppliers']
customers = db['customers']
employees = db['employees']
products = db['products']
orders = db['orders']
order_details = db['order_details']

import csv
from pymongo.client_session import ClientSession
from pymongo.collection import Collection

def populate_data(session: ClientSession, collection: Collection, csv_file: str, name: str):
    with open(csv_file, encoding="utf-8") as file:
        reader = csv.reader(file)
        rows = []
        for i, row in enumerate(reader):
            if i == 0:
                continue
            transformed_row = RowOperations(name).converted(row)
            rows.append(transformed_row)

        collection.insert_many(rows, session=session)

def populate_orders(session: ClientSession, collection: Collection, csv_file_path):
    with open(csv_file_path, encoding="utf-8") as file:
        reader = csv.DictReader(file)
        rows_out = []
        embedded = []
        order_details_keys = ["ProductID", "UnitPrice", "Quantity", "Discount"]
        prev_row = {}
        new_row = {}
        for i, row in enumerate(reader):
            print(row)
            if i >= 10:
                break
            if not prev_row:
                prev_row = row
            if row["OrderID"] != prev_row["OrderID"]:
                rows_out.append(new_row)
                embedded = []
            new_row = {k: v for k, v in row.items() if k not in order_details_keys}
            new_row = RowOperations("orders").converted(new_row.values())
            embedded_record = RowOperations("order_details").converted([row[k] for k in order_details_keys if k in row])
            embedded.append(embedded_record)
            new_row["OrderDetails"] = embedded
        collection.insert_many(rows_out, session=session)
        return


In [95]:
with client.start_session() as session:
    for file in ["categories", "customers", "employees", "orders", "products", "suppliers"]:
        file_path = "../data/" + file + ".csv"
        if file == "orders": # Handle embedded data separately
            populate_orders(session, db[file], file_path)
        else:
            populate_data(session, db[file], file_path, file)


{'OrderID': '10248', 'CustomerID': 'VINET', 'EmployeeID': '5', 'OrderDate': '1996-07-04', 'RequiredDate': '1996-08-01', 'ShippedDate': '1996-07-16', 'ShipVia': '3', 'Freight': '32.38', 'ShipName': 'Vins et alcools Chevalier', 'ShipAddress': "59 rue de l'Abbaye", 'ShipCity': 'Reims', 'ShipRegion': '', 'ShipPostalCode': '51100', 'ShipCountry': 'France', 'ProductID': '11', 'UnitPrice': '14', 'Quantity': '12', 'Discount': '0'}
{'OrderID': '10248', 'CustomerID': 'VINET', 'EmployeeID': '5', 'OrderDate': '1996-07-04', 'RequiredDate': '1996-08-01', 'ShippedDate': '1996-07-16', 'ShipVia': '3', 'Freight': '32.38', 'ShipName': 'Vins et alcools Chevalier', 'ShipAddress': "59 rue de l'Abbaye", 'ShipCity': 'Reims', 'ShipRegion': '', 'ShipPostalCode': '51100', 'ShipCountry': 'France', 'ProductID': '42', 'UnitPrice': '9.8', 'Quantity': '10', 'Discount': '0'}
{'OrderID': '10248', 'CustomerID': 'VINET', 'EmployeeID': '5', 'OrderDate': '1996-07-04', 'RequiredDate': '1996-08-01', 'ShippedDate': '1996-07-1

In [96]:
import pprint

with orders.find() as cursor:
    for i, doc in enumerate(cursor):
        if i >= 10:
            break
        pprint.pprint(doc)

{'CustomerID': 'VINET',
 'EmployeeID': 5,
 'Freight': 32.38,
 'OrderDate': '1996-07-04',
 'OrderDetails': [{'Discount': 0.0,
                   'ProductID': 11,
                   'Quantity': 12.0,
                   'UnitPrice': 14.0},
                  {'Discount': 0.0,
                   'ProductID': 42,
                   'Quantity': 10.0,
                   'UnitPrice': 9.8},
                  {'Discount': 0.0,
                   'ProductID': 72,
                   'Quantity': 5.0,
                   'UnitPrice': 34.8}],
 'OrderID': 10248,
 'RequiredDate': '1996-08-01',
 'ShipAddress': "59 rue de l'Abbaye",
 'ShipCity': 'Reims',
 'ShipCountry': 'France',
 'ShipName': 'Vins et alcools Chevalier',
 'ShipPostalCode': '51100',
 'ShipRegion': '',
 'ShipVia': 3,
 'ShippedDate': '1996-07-16',
 '_id': ObjectId('693f6a5ca5c20a6092accb57')}
{'CustomerID': 'TOMSP',
 'EmployeeID': 6,
 'Freight': 11.61,
 'OrderDate': '1996-07-05',
 'OrderDetails': [{'Discount': 0.0,
                   'Product

In [97]:

# mapping the old CSV IDs to the new MongoDB ObjectIds - to replace foreign keys with ObjectId references
category_map = {}
for cat in categories.find():
    category_map[cat['CategoryID']] = cat['_id']

supplier_map = {}
for sup in suppliers.find():
    supplier_map[sup['SupplierID']] = sup['_id']

customer_map = {}
for cust in customers.find():
    customer_map[cust['CustomerID']] = cust['_id']

employee_map = {}
for emp in employees.find():
    employee_map[emp['EmployeeID']] = emp['_id']

product_map = {}
for prod in products.find():
    product_map[prod['ProductID']] = prod['_id']

print(f"Categories: {len(category_map)}")
print(f"Suppliers: {len(supplier_map)}")
print(f"Customers: {len(customer_map)}")
print(f"Employees: {len(employee_map)}")
print(f"Products: {len(product_map)}")

# ============================================
# Step 2: Update products with references
# ============================================

# Products reference both suppliers and categories
# We replace the integer IDs with ObjectIds

for prod in products.find():
    updates = {}

    if 'SupplierID' in prod and prod['SupplierID'] in supplier_map:
        updates['SupplierID'] = supplier_map[prod['SupplierID']]

    if 'CategoryID' in prod and prod['CategoryID'] in category_map:
        updates['CategoryID'] = category_map[prod['CategoryID']]

    if updates:
        products.update_one({'_id': prod['_id']}, {'$set': updates})

print("\nUpdated products with supplier and category references")

# ============================================
# Step 3: Update employees with manager references
# ============================================

# Employees have a self-referencing relationship (ReportsTo)

for emp in employees.find():
    if 'ReportsTo' in emp and emp['ReportsTo'] and emp['ReportsTo'] in employee_map:
        employees.update_one(
            {'_id': emp['_id']},
            {'$set': {'ReportsTo': employee_map[emp['ReportsTo']]}}
        )

print("Updated employee reporting structure")
# ============================================
# Step 4: Update orders with customer and employee references
# ============================================

# Orders already have embedded OrderDetails from CSV import
# We just need to update the CustomerID and EmployeeID references

print("Updating order references...")

for order in orders.find():
    updates = {}

    # Update CustomerID reference
    if 'CustomerID' in order and order['CustomerID'] in customer_map:
        updates['CustomerID'] = customer_map[order['CustomerID']]

    # Update EmployeeID reference
    if 'EmployeeID' in order and order['EmployeeID'] in employee_map:
        updates['EmployeeID'] = employee_map[order['EmployeeID']]

    # Update ProductID references in embedded OrderDetails
    if 'OrderDetails' in order:
        updated_details = []
        for detail in order['OrderDetails']:
            if 'ProductID' in detail and detail['ProductID'] in product_map:
                detail['ProductID'] = product_map[detail['ProductID']]
            updated_details.append(detail)
        updates['OrderDetails'] = updated_details

    if updates:
        orders.update_one({'_id': order['_id']}, {'$set': updates})

print("Updated order references")

# Verifying

sample_prod = products.find_one()
print(f"\nSample product: {sample_prod['ProductName']}")
if 'SupplierID' in sample_prod:
    supplier = suppliers.find_one({'_id': sample_prod['SupplierID']})
    print(f"  Supplier: {supplier['CompanyName']}")
if 'CategoryID' in sample_prod:
    category = categories.find_one({'_id': sample_prod['CategoryID']})
    print(f"  Category: {category['CategoryName']}")

sample_order = orders.find_one()
print(f"\nSample order has {len(sample_order['OrderDetails'])} items")
first_item = sample_order['OrderDetails'][0]
if 'ProductID' in first_item:
    prod = products.find_one({'_id': first_item['ProductID']})
    print(f"  First item: {prod['ProductName']}")

print("\nAll relationships created successfully!")

Categories: 8
Suppliers: 29
Customers: 91
Employees: 9
Products: 77

Updated products with supplier and category references
Updated employee reporting structure
Updating order references...
Updated order references

Sample product: Chai
  Supplier: Specialty Biscuits, Ltd.
  Category: Beverages

Sample order has 3 items
  First item: Queso Cabrales

All relationships created successfully!


1B) List all product names and unit prices supplied by each company (supplier), along with the supplier's name. 
The output should have one document per supplier, with the required information.

This query uses a `$lookup` aggregation stage to join the suppliers collection with the products collection based on the SupplierID reference (which we converted to ObjectIds). The `$lookup` operator works similarly to a SQL JOIN, matching each supplier's `_id` with the `SupplierID` foreign key in products. I then used `$project` with `$map` to reshape the output, extracting only the ProductName and UnitPrice fields from each matched product. The final result gives one document per supplier containing an array of all their products with names and prices, demonstrating how MongoDB handles one-to-many relationships through references.


In [98]:
# 1B) List all product names and unit prices by supplier

pipeline = [
    {
        "$lookup": {
            "from": "products",
            "localField": "_id",
            "foreignField": "SupplierID",
            "as": "Products"
        }
    },
    {
        "$project": {
            "_id": 0,
            "SupplierName": "$CompanyName",
            "Products": {
                "$map": {
                    "input": "$Products",
                    "as": "prod",
                    "in": {
                        "ProductName": "$$prod.ProductName",
                        "UnitPrice": "$$prod.UnitPrice"
                    }
                }
            }
        }
    }
]

result = suppliers.aggregate(pipeline)

# Print results
for doc in result:
    print(f"\nSupplier: {doc['SupplierName']}")
    print(f"Products ({len(doc['Products'])}):")
    for prod in doc['Products']:
        print(f"  - {prod['ProductName']}: ${prod['UnitPrice']}")


Supplier: Exotic Liquids
Products (2):
  - Chang: $19.0
  - Aniseed Syrup: $10.0

Supplier: New Orleans Cajun Delights
Products (4):
  - Chef Anton's Cajun Seasoning: $22.0
  - Chef Anton's Gumbo Mix: $21.35
  - Louisiana Fiery Hot Pepper Sauce: $21.05
  - Louisiana Hot Spiced Okra: $17.0

Supplier: Grandma Kelly's Homestead
Products (3):
  - Grandma's Boysenberry Spread: $25.0
  - Uncle Bob's Organic Dried Pears: $30.0
  - Northwoods Cranberry Sauce: $40.0

Supplier: Tokyo Traders
Products (3):
  - Mishi Kobe Niku: $97.0
  - Ikura: $31.0
  - Longlife Tofu: $10.0

Supplier: Cooperativa de Quesos 'Las Cabras'
Products (2):
  - Queso Cabrales: $21.0
  - Queso Manchego La Pastora: $38.0

Supplier: Mayumi's
Products (3):
  - Konbu: $6.0
  - Tofu: $23.25
  - Genen Shouyu: $13.0

Supplier: Pavlova, Ltd.
Products (5):
  - Pavlova: $17.45
  - Alice Mutton: $39.0
  - Carnarvon Tigers: $62.5
  - Vegie-spread: $43.9
  - Outback Lager: $15.0

Supplier: Specialty Biscuits, Ltd.
Products (5):
  - C

ALTERNATIVE

In [99]:
# 1B) List all product names and unit prices by supplier

# Using a pipeline to extract product and supplier information
# The SupplierID is the foreign key connecting products to suppliers

pipeline = [
    {
        # Join suppliers and products collections
        "$lookup": {
            "from": "suppliers",
            "localField": "SupplierID",
            "foreignField": "_id",
            "as": "supplier_info"
        }
    },
    {
        # Flatten the supplier array to access supplier details
        "$unwind": "$supplier_info"
    },
    {
        # Select only the fields we need
        "$project": {
            "_id": 0,
            "ProductName": 1,
            "UnitPrice": 1,
            "SupplierName": "$supplier_info.CompanyName"
        }
    },
    {
        # Sort by supplier name for readability
        "$sort": { "SupplierName": 1 }
    }
]

result = products.aggregate(pipeline)

# Print each product with its supplier
for item in result:
    print(f"Product: {item['ProductName']}, Price: ${item['UnitPrice']}, Supplier: {item['SupplierName']}")

Product: Chartreuse verte, Price: $18.0, Supplier: Aux joyeux ecclésiastiques
Product: Côte de Blaye, Price: $263.5, Supplier: Aux joyeux ecclésiastiques
Product: Laughing Lumberjack Lager, Price: $14.0, Supplier: Bigfoot Breweries
Product: Steeleye Stout, Price: $18.0, Supplier: Bigfoot Breweries
Product: Sasquatch Ale, Price: $14.0, Supplier: Bigfoot Breweries
Product: Queso Manchego La Pastora, Price: $38.0, Supplier: Cooperativa de Quesos 'Las Cabras'
Product: Queso Cabrales, Price: $21.0, Supplier: Cooperativa de Quesos 'Las Cabras'
Product: Escargots de Bourgogne, Price: $13.25, Supplier: Escargots Nouveaux
Product: Chang, Price: $19.0, Supplier: Exotic Liquids
Product: Aniseed Syrup, Price: $10.0, Supplier: Exotic Liquids
Product: Mozzarella di Giovanni, Price: $34.8, Supplier: Formaggi Fortini s.r.l.
Product: Gorgonzola Telino, Price: $12.5, Supplier: Formaggi Fortini s.r.l.
Product: Mascarpone Fabioli, Price: $32.0, Supplier: Formaggi Fortini s.r.l.
Product: Tarte au sucre, Pr

1C) For each customer, list all the products they have purchased. The output should contain the customer information, along with a list of purchased products with product name, total quantity purchased, and total spent (computed from the list of products).


This query required traversing multiple relationships: customers → orders → order details → products. I used a series of `$lookup` operations to join these collections together, followed by `$unwind` operators to flatten the nested arrays (orders and OrderDetails) so I could access individual items. The pipeline then groups by both customer and product using `$group`, calculating the total quantity purchased and total spent per product using `$sum` and `$multiply` aggregation operators. A second `$group` stage collects all products for each customer into a single array using `$push`. This demonstrates MongoDB's ability to handle complex multi-level joins and aggregations across both embedded documents (OrderDetails) and referenced collections (customers, products).


In [100]:
# This requires joining customers -> orders -> order details -> products
# We use multiple $lookup operations to traverse these relationships

pipeline = [
    {
        # Join customers with their orders
        "$lookup": {
            "from": "orders",
            "localField": "_id",
            "foreignField": "CustomerID",
            "as": "customer_orders"
        }
    },
    {
        # Only include customers who have placed orders
        "$match": {
            "customer_orders": { "$ne": [] }
        }
    },
    {
        # Flatten the orders array
        "$unwind": "$customer_orders"
    },
    {
        # Flatten the OrderDetails array within each order
        "$unwind": "$customer_orders.OrderDetails"
    },
    {
        # Join with products to get product names
        "$lookup": {
            "from": "products",
            "localField": "customer_orders.OrderDetails.ProductID",
            "foreignField": "_id",
            "as": "product_info"
        }
    },
    {
        # Flatten product info
        "$unwind": "$product_info"
    },
    {
        # Group by customer and product to calculate totals
        "$group": {
            "_id": {
                "CustomerID": "$_id",
                "CompanyName": "$CompanyName",
                "ProductID": "$customer_orders.OrderDetails.ProductID",
                "ProductName": "$product_info.ProductName"
            },
            "TotalQuantity": { 
                "$sum": "$customer_orders.OrderDetails.Quantity" 
            },
            "TotalSpent": { 
                "$sum": { 
                    "$multiply": [
                        "$customer_orders.OrderDetails.Quantity",
                        "$customer_orders.OrderDetails.UnitPrice"
                    ]
                }
            }
        }
    },
    {
        # Group again by customer to collect all their products
        "$group": {
            "_id": "$_id.CustomerID",
            "CompanyName": { "$first": "$_id.CompanyName" },
            "ProductsPurchased": {
                "$push": {
                    "ProductName": "$_id.ProductName",
                    "TotalQuantity": "$TotalQuantity",
                    "TotalSpent": "$TotalSpent"
                }
            }
        }
    },
    {
        # Sort by company name
        "$sort": { "CompanyName": 1 }
    }
]

result = customers.aggregate(pipeline)

# Print results
for customer in result:
    print(f"\nCustomer: {customer['CompanyName']}")
    print(f"Products purchased: {len(customer['ProductsPurchased'])}")
    for product in customer['ProductsPurchased']:
        print(f"  - {product['ProductName']}")
        print(f"    Total Quantity: {product['TotalQuantity']}")
        print(f"    Total Spent: ${product['TotalSpent']:.2f}")


Customer: Hanari Carnes
Products purchased: 3
  - Jack's New England Clam Chowder
    Total Quantity: 10.0
    Total Spent: $77.00
  - Louisiana Fiery Hot Pepper Sauce
    Total Quantity: 15.0
    Total Spent: $252.00
  - Manjimup Dried Apples
    Total Quantity: 35.0
    Total Spent: $1484.00

Customer: Toms Spezialitäten
Products purchased: 2
  - Manjimup Dried Apples
    Total Quantity: 40.0
    Total Spent: $1696.00
  - Tofu
    Total Quantity: 9.0
    Total Spent: $167.40

Customer: Victuailles en stock
Products purchased: 1
  - Gustaf's Knäckebröd
    Total Quantity: 6.0
    Total Spent: $100.80

Customer: Vins et alcools Chevalier
Products purchased: 3
  - Queso Cabrales
    Total Quantity: 12.0
    Total Spent: $168.00
  - Singaporean Hokkien Fried Mee
    Total Quantity: 10.0
    Total Spent: $98.00
  - Mozzarella di Giovanni
    Total Quantity: 5.0
    Total Spent: $174.00


1D)

This query creates a materialised view by using the $merge operator at the end of an aggregation pipeline, which stores the computed results as a permanent collection rather than just returning temporary results. 

The pipeline joins customers with orders, order details, products, and categories to present the characteristics for each customer: total number of orders, total quantity purchased across all products, total revenue, a list of unique product categories ($addToSet to avoid duplicates), and the first and most recent order dates (using $min and $max). 

By materialising these pre-computed aggregations into a separate collection, subsequent queries for customer summaries become much faster since the complex calculations don't need to be repeated each time, which is particularly useful for reporting and analytics in large databases.

In [101]:
# 1D) Create materialised view: customer_sales_summary

# This pipeline aggregates customer purchasing data across all orders, storing results as a new collection

pipeline = [
    {
        # Join customers with their orders
        "$lookup": {
            "from": "orders",
            "localField": "_id",
            "foreignField": "CustomerID",
            "as": "customer_orders"
        }
    },
    {
        # Only include customers who have orders
        "$match": {
            "customer_orders": { "$ne": [] }
        }
    },
    {
        # Flatten orders array
        "$unwind": "$customer_orders"
    },
    {
        # Flatten order details within each order
        "$unwind": "$customer_orders.OrderDetails"
    },
    {
        # Join with products to get category information
        "$lookup": {
            "from": "products",
            "localField": "customer_orders.OrderDetails.ProductID",
            "foreignField": "_id",
            "as": "product_info"
        }
    },
    {
        "$unwind": "$product_info"
    },
    {
        # Join with categories to get category names
        "$lookup": {
            "from": "categories",
            "localField": "product_info.CategoryID",
            "foreignField": "_id",
            "as": "category_info"
        }
    },
    {
        "$unwind": "$category_info"
    },
    {
        # Group by customer to calculate summary statistics
        "$group": {
            "_id": "$_id",
            "CompanyName": { "$first": "$CompanyName" },
            "ContactName": { "$first": "$ContactName" },
            "Country": { "$first": "$Country" },
            "TotalOrders": { "$sum": 1 },
            "TotalQuantityPurchased": { 
                "$sum": "$customer_orders.OrderDetails.Quantity" 
            },
            "TotalRevenue": { 
                "$sum": { 
                    "$multiply": [
                        "$customer_orders.OrderDetails.Quantity",
                        "$customer_orders.OrderDetails.UnitPrice"
                    ]
                }
            },
            "Categories": { 
                "$addToSet": "$category_info.CategoryName" 
            },
            "FirstOrderDate": { 
                "$min": "$customer_orders.OrderDate" 
            },
            "MostRecentOrderDate": { 
                "$max": "$customer_orders.OrderDate" 
            }
        }
    },
    {
        # Sort by total revenue descending
        "$sort": { "TotalRevenue": -1 }
    },
    {
        # Materialize the view by storing results in new collection
        "$merge": {
            "into": "customer_sales_summary",
            "whenMatched": "replace",
            "whenNotMatched": "insert"
        }
    }
]


customers.aggregate(pipeline)

print("Materialized view 'customer_sales_summary' created successfully!")

# Ensure the result is stored as a materialised view collection:
print(f"\nTotal customers in summary: {db.customer_sales_summary.count_documents({})}")



Materialized view 'customer_sales_summary' created successfully!

Total customers in summary: 4
